# Selenium scraper for multiple websites which let you search by ZIP

## pip installs libraries.
note: %%capture suppresses output

In [1]:
%%capture
pip install selenium

In [2]:
%%capture
pip install webdriver-manager

## Define parameters.

In [55]:
STATE_TO_SEARCH = None #'CO'
    # Options include 'CO' and None, which searches all 50 states.

SITES_TO_SEARCH = ['Redfin','Rent']
    # Options include 'Rent.' and 'Redfin'
SITE_OPTIONS = ['Rent.','Redfin'] # that can be parsed

READ_ONLY = False
    # When true no data is downloaded

PARSE_WHILE_SCRAPING = True
SAVE_RAW_HTML = False # not reccomended
SAVE_ZIP_CSVs = True # highly reccomended

SAVE_BY_WEEK = True
    # instead of saving day of scrape

STARTING_ZIP = None
    # Lets you search country over multiple days.

WAIT_TIME_DICT = {'CONSTANT':{'Realtor':10,'Rent.':0,'other': 0},
                     # whether or not site redirects to CAPTCHA
                  'FAILURE' :{'other':23}
                     # after site redirects to CAPTCHA
            }

ASSUMPTION = {'PRICE':1,
              'SQFT':1,
              'BEDS':0,
              'BATHS':0
}
    #When price or sqft is range, what do we assume is the value?
        # 0 means minimum
        # 1 means maximum
        # 0.5 would split the difference

## Import libraries.

In [2]:
#scraping
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager 
from webdriver_manager.firefox import GeckoDriverManager
#data storage
import pandas # for reading the txt with ZIPs
from datetime import date # for file names
import os # for file interactions
from bs4 import BeautifulSoup as Soup # for knowing how many pages there are per ZIP
from time import sleep
from tqdm import tqdm # progress bar
import requests # for headless version
from numpy import NaN
LONG_DASH, LONGER_DASH = '–', '—'

### Import ZIPs and MHAs

In [52]:
MHAs = pandas.read_csv("sorted_zipmha22.txt",
                     delimiter = " ", names = ["ZIP","MHA"],
                    dtype = {'ZIP':str})
MHAs['State'] = MHAs.apply(lambda row: row['MHA'][:2], axis = 1)
if STATE_TO_SEARCH:
    assert type(STATE_TO_SEARCH) == str
    MHAs = MHAs[MHAs['State'] == STATE_TO_SEARCH]
ZIPs = list(MHAs['ZIP'])
ZIPs = [str(ZIP).zfill(5) for ZIP in ZIPs]
if STARTING_ZIP:
    startIndex = ZIPs.index(str(STARTING_ZIP))
    ZIPs = ZIPs[startIndex:] + ZIPs[:startIndex]
print("There are",len(ZIPs),"ZIPs to scrape")
MHAs.head()

There are 41070 ZIPs to scrape


,ZIP,MHA,State
0,00501,NY218,NY
1,00544,NY218,NY
2,00601,XX499,XX
3,00602,XX499,XX
4,00603,XX499,XX


### Install Selenium Driver Manager

In [4]:
PATH = ChromeDriverManager().install()
#PATH = GeckoDriverManager().install()

# Define functions.

In [5]:
def WAIT_TIME(context, site = 'other'):
    if site in WAIT_TIME_DICT[context].keys():
        return WAIT_TIME_DICT[context][site]
    else:
        return WAIT_TIME_DICT[context]['other']
def wait(context, site = 'other'):
    sleep(WAIT_TIME(context, site = site))

### Generate URL

In [6]:
# /appartments includes houses for rent
def URL(site, ZIP, page = 1):
    assert type(site) == str
    assert type(ZIP ) == str
    return {'Realtor':"https://www.realtor.com/apartments/{}/pg-{}".format(ZIP, page),
            'Rent.'  :"https://www.rent.com/zip-{}-apartments?page={}".format(ZIP, page),
            #'Zillow' :"https://www.zillow.com/homes/{}_r/".format(ZIP),
            'trulia' :"https://www.trulia.com/for_rent/{}_zip/".format(ZIP),
            'Redfin' :"https://www.redfin.com/zipcode/{}/apartments-for-rent/page-{}".format(ZIP, page)
           }[site]

### Create folders and return a file path to write a file to

In [7]:
STATES_GET_SEPARATE_FOLDERS = False
def makeFilePath(site, ZIP = None, fileType = 'HTML', dateOf = date.today(), page = '1'):
    for var in [site, page, fileType]: assert type(var) == str, f"{var} is not a string"
    assert fileType in ['HTML','ZIP CSV','Compiled CSV','Processed CSV']
    filePathParts = ['Scraped_Data']
    
    if STATES_GET_SEPARATE_FOLDERS:
        if STATE_TO_SEARCH: # if searching just one state
            assert type(STATE_TO_SEARCH) == str
            filePathParts.append(STATE_TO_SEARCH)
    
    filePathParts.append(fileType+'s') # FILE -> FILEs
    
    filePathParts.append(site.replace('.',''))
    
    dateString = str(dateOf.isocalendar()[:2]) if SAVE_BY_WEEK else str(dateOf)
    if not 'Compiled' in fileType: filePathParts.append(dateString)

    if fileType == 'HTML':
        filePathParts.append(ZIP)
        fileName = page.zfill(2)
        if not ('.' in fileName): fileName += '.html'
    elif 'CSV' in fileType:
        if 'ZIP' in fileType:
            assert type(ZIP) == str
            fileName = ZIP
        elif 'Compiled' in fileType: fileName = dateString
        if not ('.' in fileName): fileName += '.CSV'
    
    filePath = ""
    for filePathPart in filePathParts:
        filePath = filePath + filePathPart + '/'
        if not os.path.exists(filePath): os.mkdir(filePath)
    
    filePath += fileName
    return filePath

In [8]:
makeFilePath('Test','12345', fileType = 'HTML')#'Compiled CSV')

'Scraped_Data/HTMLs/Test/(2022, 48)/12345/01.html'

In [9]:
def HTML_to_file(HTML, filePath):
    # Does not check if file already exists.
        # Only run if file does not exists or should be written over.
    with open(filePath, 'w', encoding="utf-8") as f:
        f.write(HTML)
        f.close()

In [10]:
HTML_MIN_SIZE = 100 # KB
HTML_MIN_SIZE *= 1000 #KB -> B

def isValidHTML(filePath):
    if os.path.exists(filePath):
        if os.path.getsize(filePath) >= HTML_MIN_SIZE:
            return True
    # else:
        return False

In [11]:
def parseRedfinNumPages(pageSoup):
    pageSummary = pageSoup.find(class_="homes summary")
    if not pageSummary: return 1
    pageSummary = pageSummary.text
    if not 'of' in pageSummary: return 1
    propsPerPage = int(pageSummary.split(' ')[0])
    numPropsText = pageSummary.split('of ')[1].split(' ')[0]
    numProps = int(numPropsText)
    numPages = 1 + (numProps - 1) // propsPerPage
    if numPages > 9: numPages = 9
    return numPages
    
def parseRentNumPages(pageSoup):
    propsPerPage = 30
    numPropsTags = pageSoup.find_all(class_ = "truncate text-gray-600")
    if len(numPropsTags) <= 0: return 1
    numPropsText = numPropsTags[0].text
    numPropsText = numPropsText.split(" ")[0] # take only text before space
    numPropsText = numPropsText.replace(',','') # replace commas with nothing
    numProps = int(numPropsText)
    numPages = 1 + (numProps - 1) // propsPerPage
    return numPages

def parseNumPages(site, firstPage):
    if site not in ['Redfin', 'Rent.']: return -1
    pageSoup = Soup(firstPage) if (type(firstPage) == str) else firstPage
    return{'Redfin':parseRedfinNumPages,
            'Rent.':parseRentNumPages
    }[site](pageSoup)

### All interacitons with the Selenium Chrome driver are under the getPage function.

In [12]:
def startNewDriver():
    global driver
    driver = webdriver.Chrome(service = Service(executable_path = PATH))
        #driver = webdriver.Firefox(service = Service(executable_path = PATH))
    return driver

def getPage(URL, headless = False):
    if headless:
        HTML = requests.get(URL).content
        HTML = str(HTML)
    else:
        global driver
        try: driver.get(URL)
        except:
            print("Starting new driver.")
            driver = startNewDriver()
                
        try: HTML = driver.page_source
        except:
            return getPage(URL, headless = headless)
        if len(HTML) < HTML_MIN_SIZE:
            driver.quit()
            wait('FAILURE')
            driver = startNewDriver()
    if len(HTML) < HTML_MIN_SIZE:
        return getPage(URL, headless = headless)
    return HTML

In [13]:
def whichSite(page):
    if type(page) == str: page = Soup(page)
    for site in SITE_OPTIONS:
        if site in page.find("title").text:
            return site

In [14]:
def parsePage(page):
    pageSoup = Soup(page) if (type(page) == str) else page
    site = whichSite(pageSoup)
    if site == 'Rent.':
        columns = pageSoup.find_all(class_ = "flex flex-col gap-y-12")
        allCards = []
        for column in columns:
            someCards = column.find_all(attrs = {'data-tag_section':"free"})
            allCards.extend(someCards)
            someCards = column.find_all(attrs = {'data-tag_section':"paid"})
            allCards.extend(someCards)
        return allCards
    elif site == 'Redfin':
        allCards = []
        for section in pageSoup.find_all(class_="HomeCardsContainer flex flex-wrap"):
            allCards.extend(section)
        return allCards
    else: return -1

In [15]:
def priceUnavailable(apartment):
    for phrase in ['Contact for Price',
                  'Price Unavailable']:
        if phrase in apartment.text:
            return True
    return False # <- else

def priceAvailable(apartment):
    return not priceUnavailable(apartment)

In [16]:
def parseOneRentCard(card):
    details = card.find_all("dd", class_ = "font-normal text-body-color")
    hiddenPrice = "Price Unavailable"
    Sqftage = "Square Footage Unavailable"
    for detail in details:
        if '$' in detail.text:
            hiddenPrice = detail.text
        if 'Sqft' in detail.text: Sqftage = detail.text
    if priceAvailable(card):
        shownPrice  = card.find("p", class_ = "flex flex-1 items-center text-lg font-semibold text-black").text
        if LONG_DASH in hiddenPrice: price = hiddenPrice # price range
        else: price = shownPrice # shown price sometimes includes +
    else:
        price = "Price Unavailable"
    if 'Units Available' in card.text:
        card.find(attrs = {"data-tid":"available-units"}).text.strip().split(' ')[0]
        
    cardData = {
        'address':card.find_all(class_="overflow-hidden overflow-ellipsis whitespace-nowrap font-normal text-body-color")[0].text,
        'beds-baths':card.find_all(attrs = {'data-tid':"beds-baths"})[0].text,
        'price':price, 'square footage':Sqftage
    }
    return cardData

def parseOneRedfinCard(card):
    price = 'Price Unavailable'
    if priceAvailable(card):
        price = card.find('span', class_="homecardV2Price")
        if price:
            price = price.text
    
    beds  = 'Beds Unavailable'
    baths = 'Baths Unavailable'
    SqFt  = 'Square Footage Unavailable'
    stats = card.find_all("div", class_="stats")
    if card.find(class_="VerifiedBadge includeLabel margin-left-smaller padding-top-small"): verified = 'verified'
    else: verified = 'not verified'
    for statSoup in stats:
        stat = statSoup.text
        if 'Bed'     in stat: beds  = stat
        if 'Bath'    in stat: baths = stat
        if 'Sq. Ft.' in stat: SqFt  = stat
    try: homeType = card.find(class_="PropertyTypeDisplay font-size-smaller padding-top-small").find("span").text[1:]
    except: homeType = "Type Unknown"
    cardData = {
        'address':card.find(class_="homeAddressV2").text,
        'beds':beds, 'baths':baths, 'type':homeType,
        'price':price, 'square footage':SqFt, 'verified':verified
    }
    return cardData
def parseOneCard(site, card):
    return{'Rent.':parseOneRentCard,
          'Redfin':parseOneRedfinCard}[site](card)

In [17]:
def isValidRentApartment(card):
    bed_baths = card.find_all(attrs = {'data-tid':"beds-baths"})
    if bed_baths: return True
    else: return False
def isValidRedfinApartment(card):
    if card.find(class_="homeAddressV2"):
        return True
    else: return False
def isValidApartment(site, card):
    return{'Rent.':isValidRentApartment,
          'Redfin':isValidRedfinApartment}[site](card)

In [18]:
def parseRentCards(cards):
    parseCards('Rent.',cards)
def parseRedfinCards(cards):
    parseCards('Redfin',cards)
def parseCards(site, cards):
    apartmentData = []
    for card in cards:
        if isValidApartment(site, card):
            apartmentData.append(parseOneCard(site, card))
    #return apartmentData
    return pandas.json_normalize(apartmentData)

In [19]:
def handlePage(site, ZIP, HTML, pageSoup = None, page = '1'):
    if not pageSoup: pageSoup = Soup(HTML)
    if SAVE_RAW_HTML:
        filePath = makeFilePath(site, ZIP, page = page, fileType = 'HTML')
        HTML_to_file(HTML, filePath)
    if PARSE_WHILE_SCRAPING: return parseCards(site, parsePage(pageSoup))

def downloadZIP(site, ZIP, headless = False):
    if READ_ONLY: return pandas.DataFrame()
    # fixme doesn't work for Rent. for some reason
    # Page 1
    HTML = getPage(URL(site, ZIP), headless)
    pageSoup = Soup(HTML)
    
    DFs = [handlePage(site, ZIP, HTML, pageSoup = pageSoup)]

    numPages = parseNumPages(site, pageSoup)
    if not (numPages == -1):
        if numPages > 1:
            for page in range(2,numPages+1):
                # For pages after page 1
                page = str(page)
                HTML = getPage(URL(site, ZIP, page), headless)
                DF = handlePage(site, ZIP, HTML, page = page)
                DFs.append(DF)

    wait('CONSTANT',site = site)
    if PARSE_WHILE_SCRAPING:
        DF = pandas.concat(DFs)
        if SAVE_ZIP_CSVs:
            filePath = makeFilePath(site, ZIP, fileType = 'ZIP CSV')
            DF.to_csv(filePath, index = False)
        return DF

In [20]:
downloadZIP('Redfin','16801')

""


# Download and save everything

In [38]:
def getZIP(site, ZIP):
    filePath = makeFilePath(site, ZIP = ZIP, fileType = 'ZIP CSV')
    if os.path.exists(filePath):
        try:
            DF = pandas.read_csv(filePath)
        except:
            os.remove(filePath)
            DF = downloadZIP(site, ZIP)
    else:
        # assume HTML was not downloaded and make no attempt to find it # fixme
        DF = downloadZIP(site, ZIP)
    DFs.append(DF)

if not READ_ONLY: driver = startNewDriver()
masterDFs = {}
for site in SITES_TO_SEARCH:
    compiledFilePath = makeFilePath(site, fileType = 'Compiled CSV')
    if os.path.exists(compiledFilePath):
        print("Reading pre-compiled CSV")
        masterDF = pandas.read_csv(compiledFilePath)
    else: # no pre-existing compiled data
        DFs = [] # each DF will represent one ZIP
        for ZIP in tqdm(ZIPs):
            DF = getZIP(site, ZIP)
            DFs.append(DF)
        masterDF = pandas.concat(DFs, axis = 0)
        masterDF.to_csv(compiledFilePath)
    masterDFs[site] = masterDF
    driver.quit()

Reading pre-compiled CSV


In [39]:
for masterDF in masterDFs.values():
    print(len(masterDF))

145773


In [53]:
DF = masterDFs['Redfin']

# Derive numbers from text in DataFrame

### functions

In [56]:
def removeStudio(string):
    return string.replace('Studio–','')
def getBeds(row):
    return parseBedsBaths(row, 'beds')
def getBaths(row):
    return parseBedsBaths(row, 'baths')
def parseBedsBaths(row, output):
    assert output in ['beds', 'baths']
    string = row['beds-baths']
    string = removeStudio(string)
    if output == 'beds':
        if 'Bed' in string: return string.split('Bed')[0].strip()
        else: return('Beds Unknown')
    elif output == 'baths':
        if 'Bath' in string: return string.split('Bath')[0].split('•')[1].strip()
        else: return('Baths Unknown')
def getZIP(row):
    address = row['address']
    if type(address) == float: return NaN
    return address.split(' ')[-1]

for key in ['PRICE','SQFT']: assert ((ASSUMPTION[key] >= 0) and (ASSUMPTION[key] <= 1))
def parseRange(string, key):
    if not string: return NaN
    if type(string) == float: return string
    if ('Unavailable' in string) or (LONGER_DASH in string): return NaN
    for subString in [',', '$', '+', ' Sqft', '/mo', ' Sq. Ft.']:
        string = string.replace(subString, '')
    for dash in ['-', LONG_DASH]:
        if (dash in string):
            [minStr,maxStr] = string.split(dash)
            minVal = int(minStr)
            maxVal = int(maxStr)
            valRange = maxVal - minVal
            return minVal + valRange*ASSUMPTION[key]
    #else
    return int(string)

def getPrice(row): return parseRange(row['price'],          'PRICE')
def getSqft (row): return parseRange(row['square footage'], 'SQFT' )

### apply to DataFrame

In [59]:
if 'beds-baths' in DF.columns:
    DF['beds'] = DF.apply(getBeds, axis = 1)
    DF['baths'] = DF.apply(getBaths, axis = 1)
    DF.pop('beds-baths')
    print("beds-baths column split")
else:
    print("no column to parse")

DF['price as number'] = DF.apply(getPrice, axis = 1)
DF['ZIP'] = DF.apply(getZIP, axis = 1)
DF['sqft as number'] = DF.apply(getSqft, axis = 1)

no column to parse


In [60]:
DF = DF.merge(MHAs, on = ['ZIP'], how = 'inner').sort_values('ZIP')

In [62]:
DF.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'address', 'beds', 'baths', 'type',
       'price', 'square footage', 'verified', 'price as number', 'ZIP',
       'sqft as number', 'MHA', 'State'],
      dtype='object')

In [70]:
DF.where(DF['ZIP'] == '16801').dropna().groupby(['MHA','beds']).median()

,,Unnamed: 0.1,Unnamed: 0,price as number,sqft as number
MHA,beds,,,,


In [72]:
DF.where(DF['ZIP'] == '16801').dropna()

,Unnamed: 0.1,Unnamed: 0,address,beds,baths,type,price,square footage,verified,price as number,ZIP,sqft as number,MHA,State
